In [1]:
import warnings
warnings.simplefilter("ignore")

import scanpy as sc

"""
Only uses rna to do umap/leiden
"""

ct = 'DC'
adata = sc.read_h5ad(f'{ct}.h5ad')

adata = adata.raw.to_adata()

adata.raw = adata.copy()

adata = adata[:, adata.var.is_rna]

sc.pp.highly_variable_genes(adata, batch_key='library')

adata = adata[:, adata.var.highly_variable]

sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')

sc.external.pp.bbknn(adata, batch_key='library', n_pcs=30)
sc.tl.umap(adata)


sc.tl.leiden(adata, key_added='subset_leiden_100', resolution = 1)

adata.obs['new_subset_annotations'] = adata.obs['subset_leiden_100'].map({
    '0':'DC CD1C',
    '1':'pDC',
    '2':'DC CD1C',
    '3':'DC CD1C',
    '4':'DC CD1C ISG',
    '5':'Trash',
    '6':'DC CLEC9A',
    '7':'DC AXL',
    '8':'Trash',
})

adata.write_h5ad(f'{ct}.h5ad')
